# Problem 4.5

## part a

In [1]:
clear
clear matrix
clear mata
set matsize 11000 
set maxvar 32767
set seed 1234567
set more off

In [3]:
// loop to simulte random assignment and save to a matrix 

cap matrix drop z
matrix z=J(40, 10000, .)

qui forvalues i = 1/10000 { //create and save 50 permutations of treatment
	import delim "./data/chapter04/GerberGreenBook_Chapter4_Exercises_4-5.csv"
	tempvar teststat Z
	gen `Z' = .
	gen `teststat' = -1
	while `teststat' < 0.05{
		tempvar rannum Zri t
	    gen `rannum'=uniform()
		egen `Zri' = cut(`rannum'), group(2)
		qui reg `Zri' x
		gen `t' = _b[x]/_se[x]		
		replace `teststat' = 2*ttail(e(df_r),abs(`t'))	
	}
	replace `Z' = `Zri'
	forvalues j = 1/40 {
	matrix z[`j', `i'] = `Z'[`j']
	}
	drop _all
}

In [4]:
qui import delim "./data/chapter04/GerberGreenBook_Chapter4_Exercises_4-5.csv", clear
rename d D
rename y Y
matrix rowm = z * J(colsof(z), 1, 1/colsof(z))
matrix colnames rowm=probs
svmat double rowm, names(col)

qui gen weights = (1/probs)*D +(1/(1-probs))*(1-D)

tabstat weights, by(D) stat(v)










Summary for variables: weights
     by categories of: D (D)

       D |  variance
---------+----------
       0 |  .0004185
       1 |   .000625
---------+----------
   Total |  .0005084
--------------------


## part b

In [5]:
qui reg Y D [pw=weights]
global ate_restricted_RA = _b[D]

di "ATE (Restricted Assignment)= " $ate_restricted_RA




ATE (Restricted Assignment)= 10.712532


In [6]:
svmat z

cap matrix drop y_dis
matrix y_dis=J(10000, 1, .)

forvalues i = 1/10000 {
	tempvar weight`i'
	gen `weight`i'' = (1/probs)*z`i' +(1/(1-probs))*(1-z`i')
	qui reg Y z`i' [pw=`weight`i'']
	matrix y_dis[`i', 1] = _b[z`i']

}

In [33]:
preserve
svmat y_dis
sort y_dis1

// p value
count if abs(y_dis1) > abs($ate_restricted_RA)
di r(N)/_N
restore 



number of observations will be reset to 10000
Press any key to continue, or Break to abort
number of observations (_N) was 40, now 10,000


  56

.0056



## part c

In [34]:
qui reg Y x D [pw=weights]
global ate_cov_restricted_RA = _b[D]
di "ATE Controlling Covariance (Restricted Assignment)= " $ate_cov_restricted_RA 




ATE Controlling Covariance (Restricted Assignment)= 5.3186105


In [35]:
cap matrix drop cov_dis
matrix cov_dis=J(10000, 1, .)

forvalues i = 1/10000{
		tempvar weight`i'
		gen `weight`i'' = (1/probs)*z`i' +(1/(1-probs))*(1-z`i')
		qui reg Y x z`i' [pw=`weight`i'']		
		matrix cov_dis[`i', 1] = _b[z`i']

}

In [36]:
preserve
svmat cov_dis

// p.value controlling covariance
count if abs(cov_dis1) > abs($ate_cov_restricted_RA)
di r(N)/_N
restore 



number of observations will be reset to 10000
Press any key to continue, or Break to abort
number of observations (_N) was 40, now 10,000

  28

.0028



## part d

In [37]:
/*----------------------se_complete_RA------------------------*/

// calculate ate_complete_RA
qui reg Y D
global ate_complete_RA = _b[D]
di "ATE under Complete Assignment = " $ate_complete_RA




ATE under Complete Assignment = 10.7


In [38]:
// RI under the null ate=ate_complete_RA
cap drop Y0_sim Y1_sim Y_sim
qui gen Y0_sim = Y
qui gen Y1_sim = Y
qui gen Y_sim = .
qui replace Y0_sim = Y - $ate_complete_RA if D==1
qui replace Y1_sim = Y + $ate_complete_RA if D==0

In [39]:
capture program drop ate_complete_RA_ri
program define ate_complete_RA_ri, rclass
	replace Y_sim = Y0_sim*(1-D) + Y1_sim*(D) 
	regress Y_sim D 
    return scalar Ys_complete_RA=_b[D]	
end

tsrtest D r(Ys_complete_RA) using distout_complete_RA.dta, overwrite: ///
 ate_complete_RA_ri




Two-sample randomization test for theta=r(Ys_complete_RA) of ate_complete_RA_ri by D

Combinations:   137846528820 = (40 choose 20)
Assuming null=0
Observed theta: 10.7

Minimum time needed for exact test (h:m:s):  392479:42:00
Reverting to Monte Carlo simulation.
Mode: simulation (10000 repetitions)

progress: |........................................|

 p=0.48995 [one-tailed test of Ho:  theta(D==0)<=theta(D==1)]
 p=0.50995 [one-tailed test of Ho:  theta(D==0)>=theta(D==1)]
 p=0.48995 [two-tailed test of Ho:  theta(D==0)==theta(D==1)]

Saving log file to distout_complete_RA.dta...done.


In [40]:
// calculate se_complete_RA
preserve
use "distout_complete_RA.dta", clear
qui drop if _n==1
tabstat theta, stat(sd)
restore 






    variable |        sd
-------------+----------
       theta |  4.673591
------------------------



In [41]:
/*--------------------se_cov_complete_RA--------------------------*/

// calculate ate_cov_complete_RA
qui reg Y D x
global ate_cov_complete_RA = _b[D]
di "ATE Controlling Covariance under Complete Assignment= " $ate_cov_complete_RA




ATE Controlling Covariance under Complete Assignment= 5.3155362


In [42]:
// RI under the null ate= ate_cov_complete_RA
cap drop Y0_sim Y1_sim Y_sim
qui gen Y0_sim = Y
qui gen Y1_sim = Y
qui gen Y_sim = .
qui replace Y0_sim = Y - $ate_cov_complete_RA if D==1
qui replace Y1_sim = Y + $ate_cov_complete_RA if D==0

In [43]:
capture program drop ate_cov_complete_RA_ri
program define ate_cov_complete_RA_ri, rclass
	replace Y_sim = Y0_sim*(1-D) + Y1_sim*(D) 
	regress Y_sim D x
    return scalar Ys_cov_complete_RA=_b[D]	
end

tsrtest D r(Ys_cov_complete_RA) using distout_cov_complete_RA.dta, overwrite: ///
 ate_cov_complete_RA_ri




Two-sample randomization test for theta=r(Ys_cov_complete_RA) of ate_cov_complete_RA_ri by D

Combinations:   137846528820 = (40 choose 20)
Assuming null=0
Observed theta: 5.316

Minimum time needed for exact test (h:m:s):  408561:47:43
Reverting to Monte Carlo simulation.
Mode: simulation (10000 repetitions)

progress: |........................................|

 p=0.50495 [one-tailed test of Ho:  theta(D==0)<=theta(D==1)]
 p=0.49495 [one-tailed test of Ho:  theta(D==0)>=theta(D==1)]
 p=0.50495 [two-tailed test of Ho:  theta(D==0)==theta(D==1)]

Saving log file to distout_cov_complete_RA.dta...done.


In [44]:
// calculate se_cov_complete_RA
preserve
use "distout_cov_complete_RA.dta", clear
qui drop if _n==1
tabstat theta, stat(sd)
restore 






    variable |        sd
-------------+----------
       theta |  1.577595
------------------------



In [45]:
/*--------------------se_restricted_RA--------------------------*/

// calculate ate_restricted_RA
di "ATE under Restricted Assignment= " $ate_restricted_RA

ATE under Restricted Assignment= 10.712532


In [46]:
// RI under the null ate= ate_restricted_RA
cap drop Y0_sim Y1_sim Y_sim
qui gen Y0_sim = Y
qui gen Y1_sim = Y
qui gen Y_sim = .
qui replace Y0_sim = Y - $ate_restricted_RA if D==1
qui replace Y1_sim = Y + $ate_restricted_RA if D==0

In [57]:
// clean space
drop __00*

In [52]:
cap matrix drop distout_restricted_RA
matrix distout_restricted_RA=J(10000, 1, .)

qui forvalues i = 1/10000 {
		tempvar weight`i'
		gen `weight`i'' = (1/probs)*z`i' +(1/(1-probs))*(1-z`i')
		
		replace Y_sim = Y0_sim*(1-z`i') + Y1_sim*(z`i')
		
		qui reg Y_sim z`i' [pw=`weight`i'']
		
		matrix distout_restricted_RA[`i', 1] = _b[z`i']

}


In [53]:
/*se_restricted_RA*/
preserve
svmat distout_restricted_RA
tabstat distout_restricted_RA, stat(sd)
restore 



number of observations will be reset to 10000
Press any key to continue, or Break to abort
number of observations (_N) was 40, now 10,000


    variable |        sd
-------------+----------
distout_re~1 |  4.139767
------------------------



In [54]:
/*---------------------se_cov_restricted_RA-------------------------*/

// calculate ate_restricted_RA
di "ATE Controlling Covariance(Restricted Assignment)= " $ate_cov_restricted_RA

ATE Controlling Covariance(Restricted Assignment)= 5.3186105


In [55]:
// RI under the null ate= ate_cov_restricted_RA

cap drop Y0_sim Y1_sim Y_sim
qui gen Y0_sim = Y
qui gen Y1_sim = Y
qui gen Y_sim = .
qui replace Y0_sim = Y - $ate_cov_restricted_RA if D==1
qui replace Y1_sim = Y + $ate_cov_restricted_RA if D==0

In [57]:
// clean space
drop __00*

In [58]:
cap matrix drop distout_cov_restricted_RA 
matrix distout_cov_restricted_RA=J(10000, 1, .)


qui forvalues i = 1/10000 {
		replace Y_sim = Y0_sim*(1-z`i') + Y1_sim*(z`i')
		tempvar weight`i'
		gen `weight`i'' = (1/probs)*z`i' +(1/(1-probs))*(1-z`i')
		
		qui reg Y_sim x z`i' [pw=`weight`i'']	
		matrix distout_cov_restricted_RA[`i', 1] = _b[z`i']

}

In [59]:
/*se_restricted_RA*/
preserve
svmat distout_cov_restricted_RA 
tabstat distout_cov_restricted_RA, stat(sd)
restore 



number of observations will be reset to 10000
Press any key to continue, or Break to abort
number of observations (_N) was 40, now 10,000


    variable |        sd
-------------+----------
distout_co~1 |  1.592966
------------------------

